## <font color='green'> Least Squares Estimation using Matrix Algebra and Numerical Optimization

* Lecture note 2: p.19

In [9]:
import os
os.chdir('/Users/robbyjeffries/MSEA2022/Spring 2022/ECON 5763, Economic Analytics/Data')

In [10]:
import numpy as np 
import pandas as pd 
import math 

raw0 = pd.read_csv('College.csv')
raw0['Private']=pd.get_dummies(raw0['Private'],drop_first=True)
raw0.head()

,Unnamed: 0,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


### <font color='green'> 1) Least Squares Estimation using Matrix Algebra

https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf

In [48]:
# convert the dataframe to a numpy array (excluding the first column-college names)
raw00 = raw0.iloc[:,1:].values 

* We can suppress scientific notation using "np.set_printoptions" (e.g. np.set_printoptions(precision=2, suppress=True))
* Scientific notation: https://en.wikipedia.org/wiki/Scientific_notation
* np.set_printoptions: https://numpy.org/doc/stable/reference/generated/numpy.set_printoptions.html

In [49]:
raw00

array([[    1.,  1660.,  1232., ...,    12.,  7041.,    60.],
       [    1.,  2186.,  1924., ...,    16., 10527.,    56.],
       [    1.,  1428.,  1097., ...,    30.,  8735.,    54.],
       ...,
       [    1.,  2097.,  1915., ...,    20.,  8323.,    49.],
       [    1., 10705.,  2453., ...,    49., 40386.,    99.],
       [    1.,  2989.,  1855., ...,    28.,  4509.,    99.]])

In [13]:
# suppress scientific notation
np.set_printoptions(precision=2, suppress=True)

In [14]:
raw00

array([[    1.,  1660.,  1232., ...,    12.,  7041.,    60.],
       [    1.,  2186.,  1924., ...,    16., 10527.,    56.],
       [    1.,  1428.,  1097., ...,    30.,  8735.,    54.],
       ...,
       [    1.,  2097.,  1915., ...,    20.,  8323.,    49.],
       [    1., 10705.,  2453., ...,    49., 40386.,    99.],
       [    1.,  2989.,  1855., ...,    28.,  4509.,    99.]])

In [15]:
# Construct X matrix
# the X represents the variables used in the regression in S2
X=raw00[:,(4,0,8,11,16)] # select predictors (note that the first column was removed)
nrow = X.shape[0]
intcpt = np.ones( (nrow,1), ) # create an intercept
X = np.concatenate((intcpt, X), axis=1) # add the intercept to X (i.e X = [intcpt,X] )

In [16]:
# Construct Y vector
Y=raw00[:,15]

* The code above didn't specify whether it is a row or column vector
* Use Y=raw00[:,[15]] to specify it is a column vector

####  <font color='green'> i) Compute LS estimates

$\hat{\beta} = (X^{'}X)^{-1}X^{'}Y$
    
* inv( ) from numpy.linalg
* transpose function
* matrix multiplication

In [17]:
from numpy.linalg import inv
OLSres = inv(X.T@X)@(X.T@Y)
print(OLSres) # Compare this to the previous result obtained from the statsmodels package

[ 7.94  0.18  4.86  0.   -0.   -0.  ]


####  <font color='green'> ii) Compute the Standard Errors of the Estimates & t-statistics

$\hat{\sigma}_{\beta} = Diag \left(\sqrt{\hat{\sigma}^2(X^{'}X)^{-1}} \right)$ where $\hat{\sigma}^2 = \hat{U}^{'}\hat{U}/(n-p-1)$ and $\hat{U} = Y - X\hat{\beta}$
    
$t_{\beta} = | \hat{\beta}/\hat{\sigma}_{\beta} |$
    

In [18]:
# Calculate Residuals
resid = Y-X@OLSres
# Calculate SER (Standard error of the regression)
SER = (resid.T@resid)/(nrow-X.shape[1])
# Calculate SE
SE = np.sqrt(np.diag(SER*inv(X.T@X))) # Compare this to the previous result from the statsmodels package

In [19]:
# Calculate T statistics
Tstat = abs(OLSres/SE)

In [20]:
Tstat

array([5.57, 6.51, 4.97, 6.2 , 4.02, 0.1 ])

### <font color='green'> 2) Least Squares Estimation using Numerical Optimization

$\hat{\beta} = argmin_{\beta} (Y - X\beta)^{'}(Y - X\beta)/n$
    
* Newton Method : https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization
* Optimization in Python: https://scipy-lectures.org/advanced/mathematical_optimization/ & https://realpython.com/python-scipy-cluster-optimize/

In [21]:
from scipy import optimize

In [22]:
# Define loss fn in two ways

# loss function 1
def loss(inpt,Y,X):
    nrow=Y.shape[0]
    loss0=0

    for i in range(0,nrow):
        
            resid = Y[i]-X[i,:]@inpt
            loss0 = loss0+resid*resid
            # can be done simply: loss0+=resid*resid (add and assign)
            
    return loss0

# loss function 2
def loss2(inpt,Y,X):

    #resid = Y-X@inpt
    #loss0 = resid.T@resid
    loss0 = (Y-X@inpt).T@(Y-X@inpt)
            
    return loss0

* Optimizer "fmin" : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html

In [31]:
# Optimize
inpt = np.zeros((X.shape[1],1)) # starting value. 
# You must specify the dimension of beta, 
# which is based on the number of columns of X.
# X.shape[1] is the number of columns of X. X.shape[0] is number of rows. 
# X is nxp. Beta is px1.
OLSres2=optimize.fmin(loss2,
                      inpt, # starting value
                      args=(Y,X),
                      maxfun=40000, # sets max number of iterations before stopping. Prevents infinite runtime.
                      maxiter=40000,# sets max number of iterations before stopping. Prevents infinite runtime.
                      ftol=1e-10, # if change in function value is less that this, stop.
                      xtol=1e-10, # if the stepsize is less that this value, stop and report.
                      disp=True
                     )

Optimization terminated successfully.
         Current function value: 73023.898799
         Iterations: 1495
         Function evaluations: 2383


In [32]:
print(OLSres2)

[ 7.94  0.18  4.86  0.   -0.   -0.  ]


### <font color='darkred'> HW3
#### Robby Jeffries
##### I chose to analyze regression 2 from HW2.
* Pick one of your linear regression models in HW2 
* Compute least squares estimates, standard errors of the estimates and t-statistics <ins> using the matrix algebra and optimization algorithm as described above </ins>
* Compare them to the results previously obtained from the statsmodels package

### <font color='green'> 1) Least Squares Estimation using Matrix Algebra


In [37]:
# Construct X matrix
X2=raw00[:,(0,4,13,17)] # select predictors (note that the first column was removed)

nrow2 = X2.shape[0]

intcpt2 = np.ones( (nrow2,1), ) # create an intercept

X2 = np.concatenate((intcpt2, X2), axis=1) # add the intercept to X (i.e X = [intcpt,X] )

In [38]:
# Construct Y vector
Y2=raw00[:,15]

####  <font color='green'> i) Compute LS estimates

$\hat{\beta} = (X^{'}X)^{-1}X^{'}Y$

In [44]:
from numpy.linalg import inv

OLSres2 = inv(X2.T@X2)@(X2.T@Y2)

print(OLSres2) # Compare this to the previous result obtained from the statsmodels package

print("These values match my previous result.")

[-7.86  8.77  0.16  0.11  0.17]
These values match my previous result.


####  <font color='green'> ii) Compute the Standard Errors of the Estimates & t-statistics

$\hat{\sigma}_{\beta} = Diag \left(\sqrt{\hat{\sigma}^2(X^{'}X)^{-1}} \right)$ where $\hat{\sigma}^2 = \hat{U}^{'}\hat{U}/(n-p-1)$ and $\hat{U} = Y - X\hat{\beta}$
    
$t_{\beta} = | \hat{\beta}/\hat{\sigma}_{\beta} |$

In [40]:
# Calculate Residuals
resid2 = Y2-X2@OLSres2

# Calculate SER (Standard error of the regression)
SER2 = (resid2.T@resid2)/(nrow2-X2.shape[1])

# Calculate SE
SE2 = np.sqrt(np.diag(SER2*inv(X2.T@X2))) # Compare this to the previous result from the statsmodels package

In [46]:
print(SE2)
print("These standard errors match my previous result.")

[2.36 0.87 0.03 0.03 0.02]
These standard errors match my previous result.


In [41]:
# Calculate T statistics
Tstat2 = abs(OLSres2/SE2)

In [47]:
print(Tstat2)
print("These t-statistics match my previous result.")

[ 3.33 10.1   6.17  3.85  6.89]
These t-statistics match my previous result.
